## AccPred: Analysis Log/Report

In [1]:
library(dplyr)
library(tidyr)
library(lmSupport)
library(lmerTest)
library(ez)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Warning message:
: package ‘tidyr’ was built under R version 3.2.5Loading required package: car
Loading required package: Matrix

Attaching package: ‘Matrix’

The following object is masked from ‘package:tidyr’:

    expand

Loading required package: lme4

Attaching package: ‘lmerTest’

The following object is masked from ‘package:lme4’:

    lmer

The following object is masked from ‘package:stats’:

    step



## Prepare Data

In [2]:
dat1=read.table('accPred_PMN_MeanAmps.txt',header=T)
dat1['comp']="PMN"
dat2=read.table('accPred_N4_MeanAmps.txt',header=T)
dat2['comp']="N4"

dat=rbind(dat1,dat2)
dat['cond']=varRecode(dat$bini,c(1,2,3,4,5),c('Native Expected','Native Unexpected',
                                          'Non-native Expected','Non-native Unexpected','Control'))
dat=rename(dat,meanAmp=value)
dat=rename(dat,ppt=ERPset)
dat=separate(data=dat, col=cond,into=c('nativeness','expectedness'),sep=' ',remove=FALSE)
dat$nativeness=as.factor(dat$nativeness)
dat$expectedness=as.factor(dat$expectedness)
dat$comp=as.factor(dat$comp)
dat$cond=as.factor(dat$cond)
dat$chindex=as.factor(dat$chindex)

head(dat,2)

Warning message:
: Too few values at 360 locations: 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 109, 110, 111, 112, 113, 114, 115, 116, ...

,meanAmp,chindex,chlabel,bini,ppt,comp,cond,nativeness,expectedness
1,-3.534,4,F1,1,09_withDiff,PMN,Native Expected,Native,Expected
2,-2.724,11,FC1,1,09_withDiff,PMN,Native Expected,Native,Expected


In [3]:
head(arrange(dat,-chindex),12)

,meanAmp,chindex,chlabel,bini,ppt,comp,cond,nativeness,expectedness
1,-3.534,4,F1,1,09_withDiff,PMN,Native Expected,Native,Expected
2,-2.724,11,FC1,1,09_withDiff,PMN,Native Expected,Native,Expected
3,-1.779,12,C1,1,09_withDiff,PMN,Native Expected,Native,Expected
4,-1.314,19,CP1,1,09_withDiff,PMN,Native Expected,Native,Expected
5,-1.061,32,CPz,1,09_withDiff,PMN,Native Expected,Native,Expected
6,-3.432,38,Fz,1,09_withDiff,PMN,Native Expected,Native,Expected
7,-3.35,39,F2,1,09_withDiff,PMN,Native Expected,Native,Expected
8,-3.248,46,FC2,1,09_withDiff,PMN,Native Expected,Native,Expected
9,-2.251,47,FCz,1,09_withDiff,PMN,Native Expected,Native,Expected
10,-1.9,48,Cz,1,09_withDiff,PMN,Native Expected,Native,Expected


In [ ]:
##ROI Derivation

## Analysis
### Data Summaries

In [17]:
grandMeanByCond=dat %>% 
    group_by(cond) %>%
    summarise(meanAmp=mean(meanAmp))

head(grandMeanByCond)

,cond,meanAmp
1,Control,-0.007263333
2,Native Expected,0.1691117
3,Native Unexpected,-0.4479347
4,Non-native Expected,-0.09837056
5,Non-native Unexpected,-0.7753831


In [4]:
mod=lmer(meanAmp~nativeness*expectedness+(1|ppt),data=subset(dat,cond!="Control"),REML=F)
summary(mod)

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: meanAmp ~ nativeness * expectedness + (1 | ppt)
   Data: subset(dat, cond != "Control")

     AIC      BIC   logLik deviance df.resid 
  5867.5   5899.1  -2927.8   5855.5     1434 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.6445 -0.6316  0.0493  0.6739  3.4031 

Random effects:
 Groups   Name        Variance Std.Dev.
 ppt      (Intercept) 0.5878   0.7667  
 Residual             3.3146   1.8206  
Number of obs: 1440, groups:  ppt, 15

Fixed effects:
                                              Estimate Std. Error         df
(Intercept)                                   -0.04338    0.21999   20.40000
nativenessNon-native                          -0.14716    0.13570 1425.00000
expectednessUnexpected                        -0.62745    0.13570 1425.00000
nativenessNon-native:expectednessUnexpected   -0.07236    0.19191 1425.00000
            

In [8]:
pmn=filter(dat, comp=="PMN")
pmn=filter(pmn,cond!="Control")
pmnROI=filter(pmn,chindex%in%c(12,48,11,47,19,32))

In [34]:
pmnMod=lmer(meanAmp~nativeness*expectedness+(1+nativeness+expectedness|ppt),data=pmnROI,REML=F)
summary(pmnMod)

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: 
meanAmp ~ nativeness * expectedness + (1 + nativeness + expectedness |  
    ppt)
   Data: pmnROI

     AIC      BIC   logLik deviance df.resid 
  1083.4   1126.1   -530.7   1061.4      349 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.5534 -0.6483 -0.0306  0.5641  3.2836 

Random effects:
 Groups   Name                   Variance Std.Dev. Corr       
 ppt      (Intercept)            1.2977   1.1392              
          nativenessNon-native   1.2896   1.1356   -0.67      
          expectednessUnexpected 0.8795   0.9378   -0.85  0.48
 Residual                        0.8629   0.9289              
Number of obs: 360, groups:  ppt, 15

Fixed effects:
                                            Estimate Std. Error       df
(Intercept)                                   1.2313     0.3100  15.7800
nativenessNon-native                         -

In [12]:
n4=filter(dat, comp=="N4")
n4=filter(n4,cond!="Control")
n4ROI=filter(n4,chindex%in%c(12,48,11,47,19,32))

In [33]:
n4Mod=lmer(meanAmp~nativeness*expectedness+(1+nativeness|ppt),data=n4ROI,REML=F)
summary(n4Mod)

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: meanAmp ~ nativeness * expectedness + (1 + nativeness | ppt)
   Data: n4ROI

     AIC      BIC   logLik deviance df.resid 
  1317.5   1348.6   -650.7   1301.5      352 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-2.53165 -0.63256 -0.02299  0.66736  2.45244 

Random effects:
 Groups   Name                 Variance Std.Dev. Corr 
 ppt      (Intercept)          1.537    1.240         
          nativenessNon-native 1.086    1.042    -0.56
 Residual                      1.824    1.351         
Number of obs: 360, groups:  ppt, 15

Fixed effects:
                                            Estimate Std. Error       df
(Intercept)                                  -0.7434     0.3503  17.8000
nativenessNon-native                         -0.1278     0.3361  22.2000
expectednessUnexpected                       -0.5278     0.2013 330.0000
nativene

In [38]:
m1=lmer(meanAmp~nativeness*expectedness+(1|ppt),data=pmnROI,REML=F)
m2=lmer(meanAmp~nativeness*expectedness+(1+expectedness|ppt),data=pmnROI,REML=F)
m3=lmer(meanAmp~nativeness*expectedness+(1+nativeness|ppt),data=pmnROI,REML=F)
m4=lmer(meanAmp~nativeness*expectedness+(1+nativeness+expectedness|ppt),data=pmnROI,REML=F)
m5=lmer(meanAmp~nativeness*expectedness+(1+nativeness:expectedness|ppt),data=pmnROI,REML=F)

Warning message:
In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, : unable to evaluate scaled gradientWarning message:
In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, : Model failed to converge: degenerate  Hessian with 4 negative eigenvalues

In [36]:
anova(m1,m2,m3,m4,m5)

,Df,AIC,BIC,logLik,deviance,Chisq,Chi Df,Pr(>Chisq)
object,6,1189.125,1212.442,-588.5626,1177.125,NA,NA,NA
..1,8,1159.421,1190.51,-571.7104,1143.421,33.70422,2,4.79979e-08
..2,8,1134.55,1165.638,-559.2748,1118.55,24.87136,0,0
..3,11,1083.368,1126.115,-530.6838,1061.368,57.18198,3,2.349987e-12
..4,15,956.2346,1014.526,-463.1173,926.2346,135.133,4,3.107128e-28


In [37]:
summary(m5)

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: 
meanAmp ~ nativeness * expectedness + (1 + nativeness * expectedness |  
    ppt)
   Data: pmnROI

     AIC      BIC   logLik deviance df.resid 
   956.2   1014.5   -463.1    926.2      345 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.6055 -0.5113 -0.0107  0.5117  4.0230 

Random effects:
 Groups   Name                                        Variance Std.Dev. Corr 
 ppt      (Intercept)                                 1.5885   1.2604        
          nativenessNon-native                        3.1421   1.7726   -0.71
          expectednessUnexpected                      2.2557   1.5019   -0.80
          nativenessNon-native:expectednessUnexpected 5.1590   2.2713    0.40
 Residual                                             0.4944   0.7031        
            
            
            
  0.77      
 -0.77 -0.76
            
Number of obs:

## Centre Variables
### PMN

In [42]:
pmnROI['nat.Cent']=varRecode(pmnROI$nativeness, c('Native','Non-native'),c(.5,-.5))
pmnROI['exp.Cent']=varRecode(pmnROI$expectedness, c('Expected','Unexpected'),c(.5,-.5))

In [54]:
pm1=lmer(meanAmp~nat.Cent*exp.Cent+(1|ppt),data=pmnROI,REML=F)
pm2=lmer(meanAmp~nat.Cent*exp.Cent+(1+exp.Cent|ppt),data=pmnROI,REML=F)
pm3=lmer(meanAmp~nat.Cent*exp.Cent+(1+nat.Cent|ppt),data=pmnROI,REML=F)
pm4=lmer(meanAmp~nat.Cent*exp.Cent+(1+nat.Cent+exp.Cent|ppt),data=pmnROI,REML=F)

In [55]:
anova(m1,m2,m3,m4)

,Df,AIC,BIC,logLik,deviance,Chisq,Chi Df,Pr(>Chisq)
object,6,1189.125,1212.442,-588.5626,1177.125,NA,NA,NA
..1,8,1159.421,1190.51,-571.7104,1143.421,33.70422,2,4.79979e-08
..2,8,1134.55,1165.638,-559.2748,1118.55,24.87136,0,0
..3,11,1083.368,1126.115,-530.6838,1061.368,57.18198,3,2.349987e-12


In [57]:
summary(pm1)
summary(pm2)
summary(pm3)
summary(pm4)

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: meanAmp ~ nat.Cent * exp.Cent + (1 | ppt)
   Data: pmnROI

     AIC      BIC   logLik deviance df.resid 
  1189.1   1212.4   -588.6   1177.1      354 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.6867 -0.5740 -0.0407  0.6115  3.4021 

Random effects:
 Groups   Name        Variance Std.Dev.
 ppt      (Intercept) 0.3016   0.5492  
 Residual             1.4288   1.1953  
Number of obs: 360, groups:  ppt, 15

Fixed effects:
                         Estimate Std. Error        df t value Pr(>|t|)   
(Intercept)              -0.07449    0.18969  33.20000  -0.393  0.69706   
nat.Cent0.5               0.33286    0.17819 345.00000   1.868  0.06261 . 
exp.Cent0.5               0.53350    0.17819 345.00000   2.994  0.00295 **
nat.Cent0.5:exp.Cent0.5   0.43943    0.25199 345.00000   1.744  0.08208 . 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: meanAmp ~ nat.Cent * exp.Cent + (1 + exp.Cent | ppt)
   Data: pmnROI

     AIC      BIC   logLik deviance df.resid 
  1159.4   1190.5   -571.7   1143.4      352 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.1922 -0.5459 -0.0343  0.5305  3.6866 

Random effects:
 Groups   Name        Variance Std.Dev. Corr 
 ppt      (Intercept) 0.3014   0.549         
          exp.Cent0.5 0.8209   0.906    -0.39
 Residual             1.2146   1.102         
Number of obs: 360, groups:  ppt, 15

Fixed effects:
                         Estimate Std. Error        df t value Pr(>|t|)  
(Intercept)              -0.07449    0.18328  23.40000  -0.406   0.6881  
nat.Cent0.5               0.33286    0.16429 330.00000   2.026   0.0436 *
exp.Cent0.5               0.53350    0.28586  21.50000   1.866   0.0757 .
nat.Cent0.5:exp.Cent0.5   0.43943    0.23234 330.00000   

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: meanAmp ~ nat.Cent * exp.Cent + (1 + nat.Cent | ppt)
   Data: pmnROI

     AIC      BIC   logLik deviance df.resid 
  1134.5   1165.6   -559.3   1118.5      352 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.1772 -0.5907 -0.0685  0.6133  2.9051 

Random effects:
 Groups   Name        Variance Std.Dev. Corr 
 ppt      (Intercept) 0.6647   0.8153        
          nat.Cent0.5 1.2496   1.1178   -0.73
 Residual             1.1028   1.0501        
Number of obs: 360, groups:  ppt, 15

Fixed effects:
                         Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)              -0.07449    0.23784  18.90000  -0.313 0.757575    
nat.Cent0.5               0.33286    0.32835  19.10000   1.014 0.323394    
exp.Cent0.5               0.53350    0.15654 330.00000   3.408 0.000735 ***
nat.Cent0.5:exp.Cent0.5   0.43943    0.22139 330.

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: meanAmp ~ nat.Cent * exp.Cent + (1 + nat.Cent + exp.Cent | ppt)
   Data: pmnROI

     AIC      BIC   logLik deviance df.resid 
  1083.4   1126.1   -530.7   1061.4      349 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.5534 -0.6483 -0.0306  0.5641  3.2836 

Random effects:
 Groups   Name        Variance Std.Dev. Corr       
 ppt      (Intercept) 0.9435   0.9713              
          nat.Cent0.5 1.2896   1.1356   -0.85      
          exp.Cent0.5 0.8795   0.9378   -0.53  0.48
 Residual             0.8629   0.9289              
Number of obs: 360, groups:  ppt, 15

Fixed effects:
                         Estimate Std. Error        df t value Pr(>|t|)  
(Intercept)              -0.07449    0.26923  16.04000  -0.277   0.7856  
nat.Cent0.5               0.33286    0.32426  18.15000   1.027   0.3181  
exp.Cent0.5               0.53350    0.27894

### N4

In [62]:
n4=filter(dat, comp=="N4")
n4=filter(n4,cond!="Control")
n4ROI=filter(n4,chindex%in%c(12,48,11,47,19,32))
n4ROI['nat.Cent']=varRecode(n4ROI$nativeness, c('Native','Non-native'),c(.5,-.5))
n4ROI['exp.Cent']=varRecode(n4ROI$expectedness, c('Expected','Unexpected'),c(.5,-.5))

In [63]:
n4m1=lmer(meanAmp~nat.Cent*exp.Cent+(1|ppt),data=n4ROI,REML=F)
n4m2=lmer(meanAmp~nat.Cent*exp.Cent+(1+exp.Cent|ppt),data=n4ROI,REML=F)
n4m3=lmer(meanAmp~nat.Cent*exp.Cent+(1+nat.Cent|ppt),data=n4ROI,REML=F)
n4m4=lmer(meanAmp~nat.Cent*exp.Cent+(1+nat.Cent+exp.Cent|ppt),data=n4ROI,REML=F)

In [64]:
anova(n4m1,n4m2,n4m3,n4m4)

,Df,AIC,BIC,logLik,deviance,Chisq,Chi Df,Pr(>Chisq)
object,6,1340.798,1364.114,-664.3989,1328.798,NA,NA,NA
..1,8,1310.406,1341.495,-647.2031,1294.406,34.39161,2,3.403743e-08
..2,8,1317.485,1348.574,-650.7425,1301.485,0,0,1
..3,11,1278.14,1320.887,-628.0701,1256.14,45.34482,3,7.81592e-10


In [66]:
summary(n4m1)

Linear mixed model fit by maximum likelihood t-tests use Satterthwaite
  approximations to degrees of freedom [lmerMod]
Formula: meanAmp ~ nat.Cent * exp.Cent + (1 | ppt)
   Data: n4ROI

     AIC      BIC   logLik deviance df.resid 
  1340.8   1364.1   -664.4   1328.8      354 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.7567 -0.6057  0.0064  0.6449  2.4680 

Random effects:
 Groups   Name        Variance Std.Dev.
 ppt      (Intercept) 1.076    1.037   
 Residual             2.108    1.452   
Number of obs: 360, groups:  ppt, 15

Fixed effects:
                        Estimate Std. Error       df t value Pr(>|t|)    
(Intercept)              -1.6027     0.3085  22.5000  -5.195 3.07e-05 ***
nat.Cent0.5               0.3314     0.2164 345.0000   1.531 0.126581    
exp.Cent0.5               0.7315     0.2164 345.0000   3.380 0.000808 ***
nat.Cent0.5:exp.Cent0.5  -0.2037     0.3061 345.0000  -0.665 0.506227    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.